## Notebook for managing ArcGIS Online users

#### Find inactive users, move their content, revoke licenses, and delete 

##### By [Phil White](mailto:philip.white@colorado.edu), Earth, Environment & Geospatial Librarian, University of Colorado Boulder  

I'm on [GitHub](https:github.com/outpw)  

Other stuff [here](https:outpw.github.io)

And see [documentation for the ArcGIS API for Python](https://developers.arcgis.com/python/guide/install-and-set-up/)

In [23]:
from arcgis.gis import GIS
from arcgis.gis import ContentManager
from datetime import datetime
import csv
import os

Set your working directory. Double slashes on windows.

In [24]:
os.chdir('C:\\Users\\phwh9568\\AGOLpy')

Connect and get your user list. I named mine ucb_ago because I work at UCB

In [4]:
ucb_ago = GIS("https://ucboulder.maps.arcgis.com", "philip.white_ucboulder", "######")

Content Manager library is necessary for creating a new folder. This will be used when transfering inactive users' content.

In [5]:
cm = ContentManager(ucb_ago)

View a user's role, last log-in, account creation date, and number of content items. I will use myself:

In [6]:
me = ucb_ago.users.get('philip.white_ucboulder')

role = me.role
lastLogIn = me.lastLogin
created = me.created
itemCount = len(me.items(max_items = 1000)) #without using the max_item parameter, result will limit to 100
print(me)
print(role)
print(lastLogIn)
print(created)
print(itemCount)

<User username:philip.white_ucboulder>
org_admin
1591724328000
1462386963000
265


Note: This uses Unix epoch timestamp in miliseconds. You can make this human readable using datetime.fromtimestamp, but you must also divide the unix time by 1000 because fromtimestamp doesn't like milliseconds. ¯\\_(ツ)_/¯ Go figure.

In [7]:
Last_logIn_date = datetime.fromtimestamp(lastLogIn/1000)
Created_date = datetime.fromtimestamp(created/1000)

print("Role =", role)
print("Last Log-in = ", Last_logIn_date)
print("Created Date = ", Created_date)
print("Item Count = ", itemCount)

Role = org_admin
Last Log-in =  2020-06-09 11:38:48
Created Date =  2016-05-04 12:36:03
Item Count =  265


#### We will use created dates, last log-in dates, and item counts to determine which accounts are active and inactive.

Start by pulling the entire list of your AGO organization's users:

In [8]:
ucb_users = ucb_ago.users.search(max_users = 2000)

Use len() to tally your organization's users. Should be the same as your total users in the AGO web-interface

In [9]:
len(ucb_users) # print (ucb_users) would list every user name

1437

View accounts that have never logged in. We can find users' last log in date using .lastLogin and account creation date using .created. You can convert a date to Unix style here: https://www.epochconverter.com/. If date/time is less than 1, the user has never logged in. >0 and they have logged in at least once. 

In [10]:
NeverLoggedIn = list([])
for user in ucb_users:
    if user.lastLogin < 1:
        NeverLoggedIn.append(user.username)
        #print(user.username)
        #print(user.created)
len(NeverLoggedIn)

277

You may want to make an ignore list. Esri provides the following in their [documentation](https://developers.arcgis.com/python/guide/accessing-and-managing-users/). I left these in as an example, even though these accounts do not exist in my organization.  

You may, however, want to use an ignore list if you want to keep certain users out of this process, like administrators.

In [11]:
ignore_list = ['sitelic', 'philip.white_ucboulder'] #myself and another admin account

#### Now let's divide our users into separate lists... those we may want to delete and those we do not.

We will find the following:  
1. Users whose accounts were created before a certain time that have never logged in (inactive, **to delete**)  
2. Users whose accounts were created after a certain time that have never logged in (inactive, but new--**keep**)  
3. Users that have not logged in since before a certain time and have no content (inactive, **to delete**)  
4. Users that have logged in since a certain time and have no content (inactive, but new--**keep**)  
5. Users that have not logged in since before a certain time and have content (inactive, **delete, but will email them first and will have to transfer ownership of content**)  
6. Users that have logged in since a certain time and have content (active users, **keep**)

First, accounts created before a certain time that have never logged in. This example will find users whose accounts were created before Sept 1 2018 that have never logged in. Note: This uses time in Unix epoch in miliseconds. You can convert a date to Unix style here: https://www.epochconverter.com/. Sept 1 2018 at midnight is 1535760000000 in Unix epoch time.

In [12]:
#Accounts created prior to Sept 1 2018 that have never logged in
Created1yrago_Never = list([])
for user in ucb_users:
    if not user.username in ignore_list:
        if user.created < 1504569600000:
            if user.lastLogin < 1:
                Created1yrago_Never.append(user.username)
                #print(user.username)
                #print(user.created)

len(Created1yrago_Never) #note: I already deleted these, so zero left in this example.

0

Now let's make a list of users whose accounts were created after Sept. 1, 2018, but have never logged in. We'll keep these (for now!). 

In [13]:
#Accounts created since Sept 1 2018 that have never logged in
CreatedPastYear_Never = list([])
for user in ucb_users:
    if not user.username in ignore_list:
        if user.created > 1504569600000:
            if user.lastLogin < 1:
                CreatedPastYear_Never.append(user.username)
                #print(user.username)
                #print(user.created)

len(CreatedPastYear_Never)

277

Now let's make a list of users whose accounts are more than a year old, but have not logged in since before September 2018 and have no content. These we can delete. This may take a little time to run.

In [14]:
#Accounts created prior to Sept 1 2018, have not logged in since before Sept 1 2018 and have no content
OneYrNoLogIn_NoContent = list([])
for user in ucb_users:
    if not user.username in ignore_list:
        if user.created < 1504569600000:
            if user.lastLogin < 1504569600000:
                if len(user.items()) <1:
                    OneYrNoLogIn_NoContent.append(user.username)
                    #print(user.username)
                    #print(user.created)

len(OneYrNoLogIn_NoContent)

67

Next we'll make a list of users that have logged in since Sept. 1, 2018, but do not have content. We'll call these accounts "latent"... maybe they will create something? Maybe not! We will keep them for now,  but we can delete them next year if they remain completely inactive.  

This will take a few minutes to run.

In [15]:
#Accounts that have logged in since Sept 1 2018 and have no content
OneYrLogIn_NoContent = list([])
for user in ucb_users:
    if not user.username in ignore_list:
        if user.lastLogin > 1504569600000:
            if len(user.items()) <1:
                OneYrLogIn_NoContent.append(user.username)
                #print(user.username)
                #print(user.created)

len(OneYrLogIn_NoContent)

438

Now we'll make a list of all of the users who have not logged in since prior to September 2018 who have content. These users will potentially be deleted, but first we will email them and check if they want to keep their account. Users deleted from this list will need their content deleted or moved to another account.

In [16]:
#Accounts created prior to Sept 1 2018, have not logged in since before Sept 1 2018 and HAVE content
OneYrNoLogIn_Content = list([])
for user in ucb_users:
    if not user.username in ignore_list:
        #if user.created < 1504569600000:
        if user.lastLogin > 1:
            if user.lastLogin < 1504569600000:
                if len(user.items()) > 0:
                    OneYrNoLogIn_Content.append(user.username)
                    #print(user.username)
                    #print(user.created)

len(OneYrNoLogIn_Content)

84

Finally, we will produce a list of users that have logged in since Sept. 1, 2018 and also have content. We assume these users are active and we will not delete their accounts.  

This one will take a few minutes to run.

In [17]:
#Accounts created prior to Sept 1 2018, HAVE logged in since Sept 1 2018 and HAVE content
OneYrLogIn_Content = list([])
for user in ucb_users:
    if not user.username in ignore_list:
        if user.lastLogin > 1504569600000:
            if len(user.items()) > 0:
                OneYrLogIn_Content.append(user.username)
                #print(user.username)
                #print(user.created)

len(OneYrLogIn_Content)

569

These 6 lists should account for all of your users.   

Do the math by adding together the length (len) of all 6 lists. Notice I subtracted the ignore list)

In [18]:
allUsers = len(OneYrLogIn_Content)+len(OneYrNoLogIn_Content)+len(OneYrLogIn_NoContent)+len(OneYrNoLogIn_NoContent)+len(CreatedPastYear_Never)+len(Created1yrago_Never)

if allUsers == len(ucb_users) - len(ignore_list):
    print (allUsers)
    print ("Yay!")
else:
    print (allUsers)
    print ("... oops.")

1435
Yay!


#### Now that you've identified users to delete, you should probably send an email to the OneYrNoLogIn_Content list to double check that you can delete their accounts. 

We will write those users' names, emails, and roles to a csv. 

*Note:* I encountered a problem with pulling the first name and last name separately of certain accounts that were more than a few years old. I suppose the account creation process has changed over the years and may not have originally separated first and last name (well, that's my best guess). However, everyone has a full name, so I just grabbed that instead of doing anything more complicated.  

Also, I discovered an administrator on this list and I added an if statement that passed over any administrator accounts because I don't want to mess with those.

In [20]:
with open('OneYrNoLogIn_Content.csv', 'w', newline = '') as f: 
    writer = csv.writer(f)
    writer.writerow(['Name','User Name','Email', 'Role'])
    for user in OneYrNoLogIn_Content:
        account = ucb_ago.users.get(user)
        if account.role == 'org_admin': #Just in case there is an admin on this list, we will skip them.
            pass
        else:
            fullName = account.fullName
            email = account.email
            role = account.role
            writer.writerow([fullName, user, email, role])

Check your working directory and you should now have a csv with name and emails of all the users' whose accounts you should check before deleting.  

From here, you can do something like a mailmerge to email all of them. Anyone who wants to keep their accounts just needs to log in. Then, you can rerun the OneYrNoLogIn_Content list and they will be removed and should then appear on the OneYrLogIn_Content list that is not slated for deletion.  

*Sweet!*  

Give those people some time to respond... 

... Okay, ready?  

Let's put together our three lists of users we plan to delete into one list:

In [21]:
deleteList = OneYrNoLogIn_Content + OneYrNoLogIn_NoContent + Created1yrago_Never

len(deleteList)

151

In [25]:
print(deleteList)

['afzalan_ucboulder', 'ansm0399_ucboulder', 'aran0238_ucboulder', 'begr5871_ucboulder', 'Brian.Ferry_ucboulder', 'brian.lightfoot_ucboulder', 'Brittany.Reed_ucboulder', 'charles.ahlborn_ucboulder', 'clare.stumpf_ucboulder', 'dage9309_ucboulder', 'daro7411_ucboulder', 'deas0531_ucboulder', 'decr7697_ucboulder', 'derya.senol_ucboulder', 'Duncan.Miller_ucboulder', 'elise.gowen_ucboulder', 'elma7087_ucboulder', 'elwe2825_ucboulder', 'Emiley.Sickels_ucboulder', 'emke3075_ucboulder', 'eva.coringrato_ucboulder', 'Francisco.Perez_ucboulder', 'galen.murton_ucboulder', 'gianfranco.sotomayor_ucboulder', 'gregfauchet@gmail.com', 'Hannah.Cope_ucboulder', 'harsha.maragh_ucboulder', 'ian.w.bishop_ucboulder', 'ilas7344_ucboulder', 'jaci0087_ucboulder', 'james.fudge_ucboulder', 'jean.russell_ucboulder', 'jessica.fleck_ucboulder', 'jill.litt_ucboulder', 'Johu9721_ucboulder', 'Jordan.Kaschinske_ucboulder', 'Jose.Ortiz_1_ucboulder', 'julia.daniel_ucboulder', 'karo4560_ucboulder', 'kathryn.browning_ucbould

Double check that it is right:

In [22]:
keepList = CreatedPastYear_Never + OneYrLogIn_NoContent + OneYrLogIn_Content

if len(deleteList) == (len(ucb_users) - len(ignore_list)) - len(keepList):
    print ('ucb_users = ', len(ucb_users) - len(ignore_list))
    print (len(deleteList), "+", len(keepList), "=", (len(keepList)+len(deleteList)))
    print ('All good!')
else:
    print ('ucb_users = ', len(ucb_users))
    print (len(deleteList), "+", len(keepList), "=", (len(keepList)+len(deleteList)))
    print ('...woops')

ucb_users =  1435
151 + 1284 = 1435
All good!


## Deleting Users! Proceed with caution!

Now that you're ready to delete users, there are a few steps to be taken. Users must not have any content nor any licenses. Also, if they have items shared with a group, those items will need to be removed from the group (this is accomplished by removing the inactive user from the account. Without all of these criteria being met, you cannot delete them.  

First, if they have content, you will need to either delete it all or move it to another account. In our case, we decided to err on the side of caution. Ownership of all of this content will be transferred to our principal admin account named 'sitelic'. This is important because if people (for example, graduates) want to access web maps and apps they've created in the future, they will not be deleted and their web addresses will be preserved.  

The script below runs a loop over the delete list. For each inactive user, it will create a folder based on their username in the sitelic account content and place all of their content in that folder. Share settings will be preserved. 

First, if a user is a part of a group, it will identify items that are shared with the group and move those items first, setting their share setting to public. Then, it will look through the user's root folder and move those items to their destination folder in sitelic. Next, if the user has content in any subfolders, it will look through each folder and move those items. Empty folders will then be deleted.

Then, licenses and extensions will be revoked. The only licenses I regularly hand out are for Pro, Community Analyst, and Business Analyst. If you have other licenses you turn of for folks regularly, you may want to add them to this list. 

Finally, it will delete the user and repeat for the whole process for the next on the delete.

In the event that a user can't be deleted for some reason (either a content item couldn't be moved for some reason, or they have some other license turned on, it will skip them and print their username which you can go and investigate later. 

In [25]:
#FYI: you can view all of the potential licenses here:
ucb_ago.admin.license.all()

[<ArcGIS Insights License at https://ucboulder.maps.arcgis.com/sharing/rest/>,
 <Redistricting Online License at https://ucboulder.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Pro License at https://ucboulder.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Maps for Power BI License at https://ucboulder.maps.arcgis.com/sharing/rest/>,
 <GeoPlanner for ArcGIS License at https://ucboulder.maps.arcgis.com/sharing/rest/>,
 <AppStudio for ArcGIS License at https://ucboulder.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Community Analyst License at https://ucboulder.maps.arcgis.com/sharing/rest/>,
 <Admin Tools for ArcGIS℠ Online License at https://ucboulder.maps.arcgis.com/sharing/rest/>,
 <ArcGIS Business Analyst Web and Mobile Apps License at https://ucboulder.maps.arcgis.com/sharing/rest/>,
 <CityEngine License at https://ucboulder.maps.arcgis.com/sharing/rest/>]

In [ ]:
#Transfers a user's content to another account. 
#If a user in the delete list is an admin, they will get passed on.
#This will first check to see if the user is part of any groups and transfer any group content to the dump account, 
#updating these group items to public. Then it will remove the user from any groups. 
#Next it look through the user's root folder and move those items 
#then through each of the user's folders moving those items
#Then it will delete the user's empty folders
#Finally, it will revoke all licenses and priveleges and delete the user account.
#If a user can't be deleted for some reason it will print out the username


for userName in deleteList:
    user = ucb_ago.users.get(userName)
    if user.role == 'org_admin':
        pass
    else:
        cm.create_folder(folder = userName, owner = 'sitelic')

    
        groups = user.groups
        groupItems = []

        for group in groups:
            groupContent = group.content()
            for item in groupContent:
                if item['owner'] == user.username:
                    groupItems.append(item)
            if group.owner == user.username:
                group.reassign_to('sitelic')
            group.remove_users([user.username])

        for item in groupItems:
            try:
                item.reassign_to('sitelic', target_folder = userName)
                item.share(everyone=True)
            except:
                pass

        userContent = user.items()
        userFolders = user.folders


        for item in userContent:
            try:
                item.reassign_to('sitelic', target_folder = userName)
            except: 
                pass

        for folder in userFolders:
            folderName = (folder['title'])
            folderItems = user.items(folderName)
            for item in folderItems:
                try:
                    item.reassign_to('sitelic', target_folder = userName)
                except: 
                    pass
            cm.delete_folder(folder=folderName, owner = user.username)

        pro_license.revoke(username=userName, entitlements='*')
        community_license.revoke(username=userName, entitlements = '*')
        business_license.revoke(username=userName, entitlements = '*')

        try:
            user.delete()
        except:
            print(userName)
            pass

## Done!

Your recipient should now have a bunch of new folders containing deleted users' content. If any exceptions occurred, you can review the printed out names to investigate those accounts. 